![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcahiers-de-programmes&branch=master&subPath=/Technologie/OpenDataExamples/météorites-2.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/cahiers-de-programmes/master/bouton-callysto.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Travailler avec des données ouvertes Partie 5: Atterrissages de météorites et chutes Partie 2

Maintenant que nous avons appris à utiliser les blocs-notes Jupyter pour créer des cartes à l'aide de données géo-spatiales et à créer des widgets interactifs, analysons certaines données. Cet ensemble de données est plutôt riche et nous pouvons tirer des conclusions intéressantes des données. Pour commencer, nous devons d'abord rassembler nos bibliothèques et l'ensemble de données dans ce cahier.

In [ ]:
'''
This is exactly what we did in the previous notebook: just getting the data again
'''

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# numerical python library
import numpy as np
url = 'https://github.com/fleiser/Meteorite-landings/raw/master/Meteorite_Landings.csv'
landings = pd.read_csv(url)


# Analyse de données plus approfondie

Cet ensemble de données contient plus que la géo-localisation des chutes de météorites. Il contient également des informations sur leur présence ou non, et des quantifications telles que le type de météorite et la masse. En tant que tel, il est intéressant d’explorer davantage ces résultats, peut-être y a-t-il des tendances intéressantes dans les données - en attendant que nous les découvrions.


Les premières choses d'abord calculons le pourcentage de météorites trouvées par rapport aux météorites totales qui sont tombées

In [ ]:
a = len(landings[landings['fall'] == "Fell"])
b = len(landings[landings['fall'] == "Found"])
print("Percentage of fallen meteorites found:", b/(a+b) * 100, '%')

Il est intéressant de noter que la majorité des météorites qui tombent sont effectivement trouvées. Les météorites laissées inconnues ont-elles des propriétés distinctives? Il y a des facteurs externes tels que la géographie ou l'isolement où la météorite est tombée, ce qui la rend difficile à trouver. Mais peut-être y a-t-il aussi des facteurs internes qui contribuent à leur difficulté à trouver? Les données contiennent plusieurs propriétés telles que la masse et le type de chaque météorite. En utilisant ces propriétés, créons des visualisations pour essayer de déduire les propriétés pouvant déterminer si une météorite est située ou non.

## Histogrammes

Dans ce cas, une quantité potentielle d'intérêt est la masse des météorites qui tombent. Peut-être y a-t-il une relation entre la taille d'une météorite et sa probabilité de découverte? Créons un histogramme des masses des météorites "Found" et "Fell".

In [ ]:
# This filters down our data frame to just rows where 'fall' is 'Fell' or "Found", and then
# by typing ['mass (g)'] after wards, we're only grabbing the mass column and assigning 
# them to a new variable. 

mass_fell = landings[landings['fall'] == "Fell"]['mass (g)']
mass_found = landings[landings['fall'] == "Found"]['mass (g)']

# Here we're dropping any potential NaN values we've seen before from our columns to prevent 
# any errors when plotting. To see the error that will show, simply remove the .dropna() 

mass_fell = mass_fell.dropna()
mass_found = mass_found.dropna()

# Make a list of data to plot
plot_data = [mass_found, mass_fell]


'''
Here we create a histogram. 

bins  : This key word specifies how many bins to put the data in for the histogram

label : This is to specify the labels for each of the bars in the histogram.  
'''

# This is another way of setting the figure size. 
plt.figure(figsize=(12,8))

plt.hist(plot_data, bins = 50, label = ["Found", "Fell"])

plt.xlabel("Mass (g)", size = 16)
plt.ylabel("Counts", size = 16)
# Uncomment the line below to see a few more bars in the histogram by changing the y axes range.
# plt.ylim([0,10])
plt.legend()
plt.show()

Eh bien, c'est un histogramme particulier qui ne nous dit pas grand chose. Malheureusement, cela est la conséquence de la grande diversité de masses de météorites. Nous avons des météorites incroyablement massives, mais nous avons aussi beaucoup plus de petites météorites de masse. La diffusion des valeurs de la masse de météorites rend difficile l’assemblage des données pour créer un histogramme. Cependant, c'est une chose à laquelle nous pouvons absolument faire face! Presque chaque fois que vous traitez avec des données dont la plage est trop grande, prenez le logarithme des données pour "écraser" les données dans une plage plus condensée.

In [ ]:
# Note log10 is log base ten. Simply typing 'log' will be the natural logarithm 
# The other logarithm included in numpy is log2 for log base two. Any other logarithms
# (in the event that you need them) will have to be calculated using properties of logarithms. 

mass_fell_log = np.log10(landings[landings['fall'] == "Fell"]['mass (g)'].dropna())
mass_found_log = np.log10(landings[landings['fall'] == "Found"]['mass (g)'].dropna())

Oups, nous avons un avertissement de fonctionnement. Plus précisément, une division par zéro rencontrée dans `log10`. Cette erreur nous dit que nous avons soumis de "mauvaises" valeurs dans le logarithme ; en particulier il y a quelques météorites de masse nulle. Dans ce cas, nous allons les filtrer en ajoutant un autre cas à notre filtre où nous trouvons des météorites "Fell" et "Found". 

In [ ]:
# Here we're simply saying that the mass of the meteorite should also be greater than zero!

mass_fell_log = np.log10(landings[(landings['fall'] == "Fell") &
                                  (landings["mass (g)"] > 0)]['mass (g)'].dropna())

mass_found_log = np.log10(landings[(landings['fall'] == "Found") &
                                   (landings["mass (g)"] > 0)]['mass (g)'].dropna())


Formidable! En excluant les météorites sans masse enregistrée, nous avons corrigé notre erreur. Maintenant, passons au traçage de ces deux quantités dans un histogramme maintenant que nous avons pris le logarithme.

In [ ]:
data_for_plot = [mass_fell_log, mass_found_log]

plt.figure(figsize = (10,6))

'''
stacked = True : Tells Python we want these plots "on top of eachother". Feel free to change it to
                 False to see the difference! 
'''
plt.hist(data_for_plot, 
         bins = 20,
         stacked = True,
         label = ["Fell", "Found"])

plt.ylabel( "Counts", size = 18)
plt.xlabel("Mass of Meteorite (log$_{10}$(grams))", size = 18)
plt.legend()
plt.show()

Où nous voyons qu'il semble y avoir des différences dans les distributions. En raison de la différence d’échelle de nos comptes, il est difficile de comparer les deux distributions. Ne pas s'inquiéter! Il suffit simplement de convertir les "dénombrements" en "pourcentages" pour les mettre à la même échelle. Cela peut être considéré comme une forme de normalisation. Pour ce faire, nous devons utiliser quelques arguments cachés de la fonction histogramme `hist` de` matplotlib`.

In [ ]:
plt.figure(figsize=(11,7))

'''
Here the new arguments to hist are as follows

density  : By setting this true, this tells python to calculate "the percentage" of data within each bin
           to convert from raw counts to what can be considered a "probability density" instead. This allows
           both of our meteorite fall types to be on the same scale
          
histtype : This is a stylization parameter. "stepfilled" is simply telling Python that we want bars that look
           like "steps" and for them to be colored in. 
           
           you can also change this to ‘bar’, ‘barstacked’ or  ‘step' to see how the different plot styles 
           look. We note that some styles will affect the scaling. 

alpha    : This takes values from 0 -> 1 and are a measure of how transparent the traces are. 
'''

plt.hist(data_for_plot, 
         bins = 20, 
         density = True, 
         histtype='stepfilled', 
         alpha = 0.55,
         label = ["Found", "Fell"]) 


# The dollar signs allow us to use math symbols in the text. 
plt.xlabel("Mass of Meteorite (log$_{10}$(grams))", size = 18)
plt.ylabel("Normalized Number Counts", size = 18)
plt.title("Mass Distribution of Meteorite Observations", size = 20)

# The prop key word changes the 'proportions' of the legend. 
plt.legend(prop={'size': 16})
plt.show()

Avec cette nouvelle échelle, il semble que plus la météorite est massive, moins elle est susceptible d’être trouvée.

---

## Mise en garde
Soyez conscient de la mise à l'échelle. Il semble <_> qu'il soit beaucoup plus probable que des météorites plus massives soient moins susceptibles d’être trouvées. Mais, gardez à l'esprit que l'histogramme bleu ne représente qu'environ 2,5% de toutes les météorites observées.

---

### Interprétation

L'apparence que plus une météorite est massive est faible, moins elle est susceptible d'être trouvée, semble contre-intuitive. Ceci est en réalité le résultat à la fois de l'effet de l'atmosphère sur de grands corps en mouvement rapide et d'une conséquence de la définition de "Found" dans cet ensemble de données.

En termes d'atmosphère, les plus grosses météorites ont tendance à exploser lorsqu'elles pénètrent dans l'atmosphère terrestre. Par exemple, en 2013, un météore plutôt important a explosé au-dessus de la Russie de Tcheliabinsk, et sa chute est visible dans la vidéo YouTube ci-dessous.

In [ ]:
# This library allows us to embed YouTube in Jupyter. 
from IPython.display import YouTubeVideo

YouTubeVideo('fBLjB5qavxY',width=1024*0.75, height=576*0.75)

Malgré toutes les preuves vidéo de la chute de ce météore, si nous consultons ce météore dans notre ensemble de données, nous trouverons quelque chose d’intéressant.

In [ ]:
landings[landings.name == 'Chelyabinsk']

Malgré les nombreux angles disponibles pour voir le météore, il n'a jamais été retrouvé? Pourquoi donc? Eh bien, cela nous amène à la sémantique. Ce météore était un astéroïde d'environ 20 mètres de largeur avec une masse supérieure à 10 000 tonnes entrant dans l'atmosphère. Cependant, seulement environ 1000 kg du météore à ce jour ont été récupérés. En conséquence, ce météore est classé comme "tombé" au lieu d'être trouvé. De plus, la masse enregistrée n'est que approximativement ce qui a été récupéré jusqu'à présent.


Il y a une relation entre la masse de la météorite et si elle est trouvée. Cependant, cette relation est principalement due à la plus grande probabilité d’explosion d’une météorite de grande taille, et à la définition de «trouvé» exigeant la récupération de la majorité du corps.

Pour plus d'informations sur les explosions de météorites et les météores de Chelyabinsk, voir

1. [L'article de Wikipedia](https://fr.wikipedia.org/wiki/Superbolide_de_Tcheliabinsk)
1. [Cet article d'alerte scientifique](https://www.sciencealert.com/why-do-meteors-explode-when-they-reach-earth-atmosphere)

Certes, il existe d’autres facteurs liés à la présence de certaines météorites et d’autres, à la masse de la météorite telle que la géographie, ou si elle a été signalée ou non. Quoi qu’il en soit, en explorant la relation avec la masse, nous avons pu découvrir une tendance intéressante cachée dans les données.

# Conclusion

Dans ce cahier, nous avons montré comment vous pouvez travailler avec votre jeu de données pour obtenir des informations plus intéressantes dans le jeu de données. Plus important encore, nous avons suivi les étapes pour créer un histogramme et couvrir de nombreux problèmes potentiels que vous pourriez rencontrer en le faisant. Nous avons couvert certaines erreurs courantes et des problèmes plus subtils lorsque nous travaillions avec un ensemble de données avec un grand nombre de valeurs et certaines solutions à ces problèmes. Nous avons également expliqué comment certaines tendances intéressantes en matière de données peuvent avoir des explications parfaitement raisonnables et moins intéressantes que celles que les données peuvent nous laisser croire. Nous espérons que cette série de didacticiels vous permettra de vous sentir plus en confiance pour travailler avec des données ouvertes dans les ordinateurs portables Jupyter.

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)